This script can generate the tracking inference (.gz) file given a pytorch detection model and a video.

In [ ]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.tracking.config import Config as TrackConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints, keypoints_post_inference_processing
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os
import copy
import json
from speed_trapv3.tracking.tracking import get_video_properties, transform_image, write_to_json#, get_frame_box
from sparrow_tracky import Tracker, euclidean_distance
from sparrow_datums import AugmentedBoxTracking, BoxTracking, FrameBoxes, PType

In [ ]:
def get_box_properties(box_in):
    """Split the augmented box into numpy arrays.
    Args:
        box_in (FrameAugmentedBoxes): FrameAugmentedBoxes produced by the model.
    Returns:
        tuple: A tuple of three numpy arrays.
    """
    box = box_in
    labels = box.labels
    scores = box.scores
    box = box.array[:, 0: 4]
    return (box, scores, labels)

In [ ]:
def make_path(path_in):
    """Check if a given path exists and make one if it doesn't.

    Args:
        path_in (str): _description_ Input path that might need to be created.
    """
    if not os.path.exists(path_in):
        os.mkdir(path_in)

In [ ]:
video_path = "/code/data/datasets/common_hall/source_video/25_resampled_vid.mp4"
model_path = "/code/data/models/detection/model.pth"
temp_save_path = "/code/data/datasets/temp_imgs"

In [ ]:
detection_model = RetinaNet().eval().cuda()
detection_model.load(DetConfig.trained_model_path)

In [ ]:
fps, n_frames = get_video_properties(video_path)
reader = imageio.get_reader(video_path)
vehicle_tracker = Tracker(TrackConfig.vehicle_iou_threshold)
slug = "hard_coded"
for i in tqdm(range(n_frames)):
    data = reader.get_data(i)
    data = cv2.rectangle(
        data, (450, 200), (1280, 720), thickness=5, color=(0, 255, 0)
    )
    # img = imageio.imread(image_path)
    img_h, img_w, _ = data.shape
    aug_boxes = detection_model(data)
    aug_boxes = aug_boxes[aug_boxes.scores > TrackConfig.vehicle_score_threshold]
    boxes = aug_boxes.array[:,:4]
    vehicle_boxes = FrameBoxes(
    boxes,
    PType.absolute_tlbr,  # (x1, y1, x2, y2) in absolute pixel coordinates [With respect to the original image size]
    image_width=data.shape[1],
    image_height=data.shape[0],
    ).to_relative()  
    vehicle_tracker.track(vehicle_boxes)
make_path(str(TrackConfig.prediction_directory / slug))
vehicle_chunk = vehicle_tracker.make_chunk(fps, TrackConfig.vehicle_tracklet_length)
vehicle_chunk.to_file(
    TrackConfig.prediction_directory / slug / f"{slug}_vehicle.json.gz"
    )
    # for box in boxes:
    #     x1, y1, x2, y2 = (box * np.array([img_w, img_h, img_w, img_h])).astype(int)
    #     data = cv2.rectangle(data, (x1, y1), (x2, y2), (255, 0, 255), thickness=4)
    # im_rgb = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    # save_path = '/code/data/datasets/temp_imgs'
    # filename = str(i) + ".jpg"
    # cv2.imwrite(os.path.join(save_path, filename), im_rgb)

In [ ]:
from speed_trapv3.tracking.tracking import get_video_properties, transform_image, write_to_json
from pathlib import Path
gz_path = Path("/code/data/datasets/tracking/predictions/hard_coded/hard_coded_vehicle.json.gz")
video_path = "/code/data/datasets/common_hall/source_video/25_resampled_vid.mp4"
write_to_json(gz_path, video_path)